In [23]:
import numpy as np
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import pandas as pd

## Custom metric for KNN classifier

The basic idea of DTW is to find the best possible match between two time series by computing the distance between corresponding points. The algorithm works by creating a grid of all possible pairwise distances between the points of the two series, and then finding the optimal path through the grid that minimizes the total distance. The optimal path is found using dynamic programming, which allows for efficient computation of the optimal path.

Example:
To understand how DTW works with different time steps, let's consider an example. Suppose we have two time series, A and B, with different time steps as shown below:
``` 
A: 1, 4, 5, 8, 9, 10
B: 2, 4, 7, 8
```
The first step in DTW is to create a matrix of pairwise distances between each point in series A and series B. The matrix will have a size of m x n, where m is the length of series A and n is the length of series B.
``` 
2   0   3   7
3   1   2   6
6   2   0   4
7   3   1   0
8   4   2   1
9   5   3   2 
```
The next step is to find the optimal path through the matrix that minimizes the total distance between the two series. The optimal path is found using dynamic programming, which computes the minimum distance path through the matrix from the starting point (1,1) to the ending point (m,n). The path can move only in the right, down, or diagonally-right-down directions.    

The optimal path is shown as a thick line and corresponds to the non-linear alignment of the two time series. The total distance along this path is 2 + 1 + 0 + 0 + 1 + 2 = 6. This is the DTW distance between the two time series. 

In [2]:
# We create a separate function for the metric to be used by the KNN classifier
# https://stackoverflow.com/questions/57015499/how-to-use-dynamic-time-warping-with-knn-in-python

def DTW(a, b):   
    an = a.size
    bn = b.size
    pointwise_distance = distance.cdist(a.reshape(-1,1),b.reshape(-1,1))
    cumdist = np.matrix(np.ones((an+1,bn+1)) * np.inf)
    cumdist[0,0] = 0

    for ai in range(an):
        for bi in range(bn):
            minimum_cost = np.min([cumdist[ai, bi+1],
                                   cumdist[ai+1, bi],
                                   cumdist[ai, bi]])
            cumdist[ai+1, bi+1] = pointwise_distance[ai,bi] + minimum_cost

    return cumdist[an, bn]

In [16]:
# Rundown of what some of the lines do in the function

#toy dataset 
X = np.random.random((100,10))
y = np.random.randint(0,2, (100))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train[0])
print()
# Compute distance between each pair of the two collections of rectangular matrix m*n inputs.
point_wise = distance.cdist(X_train.reshape(-1,1), X_train.reshape(-1,1))
# cdist(a, b) -> a is a m*n matrix with m datapoints and n-dimension feature, b is q*n matrix with q datapoints and n-dimension feature
# returns an array of size m*q where each value corresponds to distance between a[ith] and b[jth] for i in range(m) and j in range(q)
print(point_wise[:5])
print()

# Matrix of infinity
an = bn = X_train.size
cumdist = np.matrix(np.ones((an+1,bn+1)) * np.inf)
cumdist[0,0] = 0

print(cumdist)

[0.34222583 0.00574065 0.65221472 0.03331665 0.4373576  0.04250133
 0.46738159 0.76390735 0.88438794 0.26647845]

[[0.         0.33648518 0.30998889 ... 0.07811217 0.30786196 0.00817625]
 [0.33648518 0.         0.64647407 ... 0.41459735 0.02862321 0.32830893]
 [0.30998889 0.64647407 0.         ... 0.23187672 0.61785086 0.31816514]
 [0.30890919 0.02757599 0.61889808 ... 0.38702136 0.00104722 0.30073294]
 [0.09513177 0.43161695 0.21485712 ... 0.0170196  0.40299373 0.10330802]]

[[ 0. inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]]


#### Link to cdist: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

In [22]:
for ai in range(an):
    for bi in range(bn):
        minimum_cost = np.min([cumdist[ai, bi+1],
                                   cumdist[ai+1, bi],
                                   cumdist[ai, bi]])
        cumdist[ai+1, bi+1] = point_wise[ai,bi] + minimum_cost
    print(np.isposinf(cumdist).sum())

375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375591
375200
374530
373860
373190
372520
371850
371180
370510
369840
369170
368500
367830
367160
366490
365820
365150
364480
363810
363140
362470
361800
361130
360460
359790
359120
358450
357780
357110
356440
355770
355100
354430

### Using actual data for custom metric

In [24]:
data_name = r'C:/Users/siddp/OneDrive/Desktop/UCR_TS_Archive_2015/UCR_TS_Archive_2015/Earthquakes/Earthquakes_train'
data_train = pd.read_csv(data_name, header=None)

data_name = r'C:/Users/siddp/OneDrive/Desktop/UCR_TS_Archive_2015/UCR_TS_Archive_2015/Earthquakes/Earthquakes_test'
data_test = pd.read_csv(data_name, header=None)

In [25]:
data_train.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,0,-0.26927,-0.26927,-0.26927,-0.26927,-0.26927,3.13690,-0.26927,-0.26927,2.98420,...,-0.26927,-0.26927,-0.26927,-0.26927,-0.26927,-0.26927,-0.26927,-0.26927,-0.26927,-0.26927
1,1,-0.46887,2.74800,1.62630,-0.46887,-0.46887,-0.46887,-0.46887,-0.46887,1.66340,...,-0.46887,1.61700,-0.46887,2.06200,-0.46887,-0.46887,-0.46887,-0.46887,1.63560,-0.46887
2,0,2.24290,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,2.20040,...,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296,-0.39296
3,0,-0.45836,2.42290,-0.45836,2.51620,1.98760,-0.45836,-0.45836,-0.45836,-0.45836,...,-0.45836,-0.45836,-0.45836,-0.45836,-0.45836,-0.45836,-0.45836,-0.45836,-0.45836,-0.45836
4,0,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,...,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609,-0.58609


In [26]:
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,1,-0.51801,-0.51801,2.65420,-0.51801,-0.51801,-0.51801,-0.51801,1.45620,2.55840,...,-0.51801,-0.51801,-0.51801,-0.51801,-0.51801,-0.51801,-0.51801,-0.51801,1.46580,-0.51801
1,0,1.94370,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311,...,2.45780,3.36560,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311,-0.35311
2,0,2.63850,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,...,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610,-0.31610
3,0,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114,...,1.36690,-0.53114,2.14740,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114,-0.53114
4,1,-0.59366,2.02010,1.17470,-0.59366,-0.59366,1.60600,1.21790,1.58880,-0.59366,...,1.22650,-0.59366,-0.59366,-0.59366,1.49390,-0.59366,-0.59366,-0.59366,1.89930,-0.59366


In [35]:
#train
# If metric is a callable function, it takes two arrays representing 1D vectors (each datapoint from dataset) as inputs and must 
# return one value indicating the distance between those vectors.
clf = KNeighborsClassifier(n_neighbors=1, metric=DTW)
clf.fit(data_train.iloc[:, 1:], data_train.iloc[:, 0:1])

c:\Users\siddp\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(metric=<function DTW at 0x000001E72148F9D0>, n_neighbors=1)

In [46]:
#evaluate
y_pred = clf.predict(data_test.iloc[:100, 1:])
print(classification_report(data_test.iloc[:100, 0:1], y_pred))